# Light Sensor

Load the base overlay, worth noting when done in the full version of our code this step is not neccesary as a custom overlay was designed in simulink

In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

Connect peripheral components to the board, firstly connect the Pynq Grove Adapter to PMODB. Secondly connect GROVE ADC port J1 (SCL,SDA, VCC, GND) to port G4 on the Grove Adapter. Lastly ensure the Grove ALS is wired up to port J2 of the GROVE ADC (GND, VCC, NC, SIG).

GROVE ADC provides a raw sample to start off with, this is then converted into a resistance value which is further converted into a temperature value. 

The light sensor must be connected to an ADC. As it is an analogue component and the PMOD grove adapter requires digital outputs.

In [ ]:
from pynq.lib.pmod import Grove_Light
from pynq.lib.pmod import PMOD_GROVE_G4 # Import constants

lgt = Grove_Light(base.PMODB, PMOD_GROVE_G4)

This section of code is used for giving the readings a variable and to also print a value to the screen to give a good idea of the value of the first point.

In [ ]:
sensor_val = lgt.read()
print(sensor_val)

Log the sensor value for every 100ms,this value gives us plenty of readings over the ten seconds the sensor is run for. Covering and uncovering the light sensor will vary the measured signal. taking a reading every 100ms for 10s gives us 100 readings. Its worth noting that the ADC gives a delay of 47us on the readings. So say for example we took a reading of 10ms over 10s we would recieve 953 readings not 1000.

r_log is a list of values. We call upon the variable lgt many times here, where lgt was described in a previous section of code as being the value the light sensor is giving off.

As can be seen in the section of code the multiple temperature values are read over a 10 second period.

To change the light intensity, cover and uncover the light sensor. In typical ambient light, there is no need to provide an external light source, as the sensor is already reading at full scale.

In [ ]:
import time

lgt.set_log_interval_ms(100)
lgt.start_log()
time.sleep(10)
r_log = lgt.get_log()

using values for min and max and the length of the list a plot can be made for the sensor values

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(range(len(r_log)), r_log, 'ro')
plt.title('Grove Light Sensor Plot')
min_r_log = min(r_log)
max_r_log = max(r_log)
plt.axis([0, len(r_log), min_r_log, max_r_log])
plt.show()

calling the variable in this manner allows the list of values to be viewed

In [ ]:
r_log

this function allows us to see the amount of values that are contained within the list

In [ ]:
len(r_log)

Using analysis techniques the probability of a number turning up can be tested. This is done for the purposes or using numerical values to try and detect which number is the most random. it can be tricky to say how random a number truly is. So a Gaussian plot is made with probability values on the y axis, if the peak for the probability value is lower was can determine there is lower probability of numbers re-occuring meaning that the number can be considered more random.

Math and statistics libraries were called upon as they made the whole process more simple. These values were considered when it came to the final sensor selection.

A multiple of 100 was put across the x-axis to show the values and the shape produced better. 

In [ ]:
import math
import statistics #library that gives us a standard deviation function

for i in range (0, len(r_log)):
    r_log[i] *= 100            #loop for range of list of values, multiplication

pi = 3.14159265359
sigma = statistics.stdev(r_log) #standard deviation value
total = 0
r_log.sort() #puts values from lowest to highest

for i in range(0, len(r_log)):
    total += r_log[i]          # obtaining the total value for all readings in the list

i = 0
mu = total/len(r_log) # mean is the total values in the list divided by how many variables are in the list
a = 1/((math.sqrt(2*pi))*sigma) #a is the value prior to the exponetial function.
        
prob_r_log = [] #creating an empty array to plot new graph with the changed values

for i in range(0, 100000): #how large the graph is, varies from sensor to sensor
    prob_r_log.insert(i, a*math.exp((-1)*(((i - mu)**2)/(2*(sigma**2))))) #filling in empty array with values relative to the 
                                                                          #probability density function equation
plt.plot(range(100000), prob_r_log, 'ro') #range
plt.title('Gaussian Test Plot') #plot title
min_prob_r_log = 0 #minimum value
max_prob_r_log = max(prob_r_log) #maximum value
plt.axis([0, 70000, min_prob_r_log, max_prob_r_log]) #size of the axis
plt.show() #showing the output plot

# Temperature Sensor

In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

Connect peripheral components to the board, firstly connect the Pynq Grove Adapter to PMODA. Secondly connect GROVE ADC port J1 (SCL,SDA, VCC, GND) to port G4 on the Grove Adapter. Lastly ensure the Grove ALS is wired up to port J2 of the GROVE ADC (GND, VCC, NC, SIG).

GROVE ADC provides a raw sample to start off with, this is then converted into a resistance value which is further converted into a temperature value. The resistance of a thermistor will increase as the ambient room temperature decreases. The range of temperature the component can detect is from -40 - 125 degrees celsius. this has a 1.5 error to be considered within this range.

The temperature sensor must be connected to an ADC. As it is an analogue component and the PMOD grove adapter requires digital output to operate.

In [ ]:
import math
from pynq.lib.pmod import Grove_TMP
from pynq.lib.pmod import PMOD_GROVE_G3 # import constants

# Grove2pmod is connected to PMODA (2)
# Grove ADC is connected to G3 (pins [6,2])
tmp = Grove_TMP(base.PMODA, PMOD_GROVE_G3)

code snippet designed to give a measured value and format a value:

In [ ]:
temperature = tmp.read()
print(float("{0:.2f}".format(temperature)),'degree Celsius')

Log the sensor value for every 100ms,this gives us 100 readings total over 10s. The value does not change greatly in ambient room conditions, making it an unideal choice for random number generation as the numbers are incredibly linear. Unless the sensor was to be put in a system which varied temperature greatly, however the more systematic a number generation is the less random it becomes.

tmp_log is a list of values. We call upon the variable tmp several times, where tmp was described as the value of the resistance increasing or decreasing in the thermistor then given as a temperature reading.

In [ ]:
import time
ms_delay = 100
delay_s = 10
tmp.set_log_interval_ms(ms_delay)
tmp.start_log()
time.sleep(delay_s) # Change input during this time
tmp_log = tmp.get_log()

Using maximum and minimum values with the matplotlib library scatter graphs with the readings are simply made

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(range(len(tmp_log)), tmp_log, 'ro')
plt.title('Grove Temperature Plot')
min_tmp_log = min(tmp_log)
max_tmp_log = max(tmp_log)
plt.axis([0, len(tmp_log), min_tmp_log, max_tmp_log])
plt.show()

same probability test as done for the previous sensor. This gives us another graph for us to compare probability values to try and give numerical evidence aswell as empirical evidence for which sensor has the most truly random values and the least user input

the multiplier range and the range of values across the x-axis is also greatly varied

In [ ]:
import math
import statistics #library that gives us a standard deviation function

for i in range (0, len(tmp_log)):
    tmp_log[i] *= 100000       #loop for range of list of values, multiplication

pi = 3.14159265359
sigma = statistics.stdev(tmp_log) #standard deviation value
total = 0
tmp_log.sort() #puts values from lowest to highest

for i in range(0, len(tmp_log)):
    total += tmp_log[i] # obtaining the total value for all readings in the list

i = 0
mu = total/len(tmp_log) # mean is the total values in the list divided by how many variables are in the list
a = 1/((math.sqrt(2*pi))*sigma) #a is the value prior to the exponetial function.
        
prob_tmp_log = [] #creating an empty array to plot new graph with the changed values

for i in range(0, 100000): #how large the graph is, varies from sensor to sensor
    prob_tmp_log.insert(i, a*math.exp((-1)*(((i - mu)**2)/(2*(sigma**2))))) #filling in empty array with values relative to the 
                                                                            #probability density function equation
plt.plot(range(100000), prob_tmp_log, 'ro') #range
plt.title('Gaussian Test Plot') #plot title
min_prob_tmp_log = 0 #min value
max_prob_tmp_log = max(prob_tmp_log) #max value
plt.axis([2400, 3000, min_prob_tmp_log, max_prob_tmp_log]) #axis range
plt.show() #show output plot

# Sound Sensor

Import the standard overlay, this changes in the final notebook where a custom overlay was used.

In [ ]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

Calling the PMOD grove adapter and ADC library. No pre-existing library existed for the sound sensor, so the ADC library was used and was succesful in converting the analogue output signal from the device and converting it into readable values. The sound sensor has a simple microphone that detects sound intensity in an ambient environment. 

Worth noting that the sound sensor gave multiple zero values. In the final notebook as loop was written to remove these values.

In [ ]:
import math
from pynq.lib.pmod import Grove_ADC
from pynq.lib.pmod import PMOD_GROVE_G3 # Import constants

# Grove2pmod is connected to PMODB (2)
# Grove ADC is connected to G3 (pins [6,2])
snd = Grove_ADC(base.PMODB, PMOD_GROVE_G3)

creating a variable for the read values given fromt the sensor, also printing a singular value

In [ ]:
snd_val = snd.read()
print(snd_val)

with a 10ms delay over a 10s reading 1000 results should be recorded. Bearing in mind that there is a 47us delay on the ADC readings of analogue values, 953 readings are taken in total over the time.

In [ ]:
import time 
ms_delay = 10
delay_s = 10
snd.set_log_interval_ms(ms_delay)
snd.start_log()
time.sleep(delay_s)
snd_log = snd.get_log()

Plotting the values of the sound sensor, using max, min and matplotlib library.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(range(len(snd_log)), snd_log, 'ro')
plt.title('Grove Sound Plot')
min_snd_log = min(snd_log)
max_snd_log = max(snd_log)
plt.axis([0, len(snd_log), min_snd_log, max_snd_log])
plt.show()

removing the numerous zero values from the list

In [ ]:
limit = len(snd_log)

for i in range(limit):
    while (i<limit):
        if (snd_log[i] == 0.0): #if a value in the list equals zero
            snd_log.remove(snd_log[i]) #remove value from the list
            limit = limit -1 #decrease the size of the limit
            continue
    
        i = i+1

plot again with removed zeroes

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(range(len(snd_log)), snd_log, 'ro')
plt.title('Grove Sound Plot')
min_snd_log = min(snd_log)
max_snd_log = max(snd_log)
plt.axis([0, len(snd_log), min_snd_log, max_snd_log])
plt.show()

Probability plot once again to compare how often numbers re-occur within the sensor readings and also the lower the probability the increased chance the number can be called random.

In [ ]:
import math
import statistics #library that gives us a standard deviation function

for i in range (0, len(snd_log)): #len functions shows how many variables are in a list
    snd_log[i] *= 100 #multiplatication is simply to make the graph looks nicer

pi = 3.14159265359
sigma = statistics.stdev(snd_log) #standard deviation function
total = 0 #starting the value at zero
snd_log.sort() #puts list into values starting from lowest to highest

for i in range(0, len(snd_log)): #from range of 0 to how many variables are in the lsit
    total += snd_log[i] #obtains total value of all the variables together in the sound readings list

i = 0
mu = total/len(snd_log) #value for mean
a = 1/((math.sqrt(2*pi))*sigma) #value for pre exponetial value
        
prob_snd_log = [] #empty arrray to be filled with appropriate values

for i in range(0, 100000):
    prob_snd_log.insert(i, a*math.exp((-1)*(((i - mu)**2)/(2*(sigma**2))))) #filling empty array with values

plt.plot(range(100000), prob_snd_log, 'ro') #size of range
plt.title('Gaussian Test Plot') #title name
min_prob_snd_log = 0 #min value
max_prob_snd_log = max(prob_snd_log) #max value
plt.axis([0, 100, min_prob_snd_log, max_prob_snd_log]) #axis plot
plt.show() #graph display

# OLED Example

Import peripherals

In [ ]:
from pynq.lib.pmod import Grove_OLED
from pynq.lib.pmod import PMOD_GROVE_G4
#connect to PMODA
pmod_oled = Grove_OLED(base.PMODA, PMOD_GROVE_G4)

using the clear commands wipes the screen of any previous text.
The write command allows text to be written onto the screen.

In [ ]:
pmod_oled.clear()
pmod_oled.write('Random Number Generation EE315 Digital Project')

In [ ]:
pmod_oled.clear()
pmod_oled.write('James Wilcox\n   Calum Parker\n   Nicholas Smith')

In [ ]:
pmod_oled.clear()